##This notebook can be used to classify the images with eddy from non-eddy using google colab platform.

###Mount directories from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Import libraries

In [ ]:
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt

In [ ]:
data_dir = ('/content/drive/your/dir/')
data_dir = pathlib.Path(data_dir)
print(data_dir)

###From 3 channel RGB to 1 channel grayscale image conversion using Keras ImageDataGenerator. (Uncomment if needed).

In [ ]:
# img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

In [ ]:
# train_ds = tf.data.Dataset.from_generator(
#     lambda: img_gen.flow_from_directory(data_dir, batch_size=32, shuffle=True),
#     output_types=(tf.float32, tf.float32))

In [ ]:
# def convert_to_grayscale(image, label):
#   return tf.image.rgb_to_grayscale(image), label ## check with actual image

In [ ]:
# images, _ = next(iter(train_ds.take(1)))
# image = images[0]
# print('Before conversion --> ', image.shape)

# train_ds = train_ds.map(convert_to_grayscale)
# images, _ = next(iter(train_ds.take(1)))

# image = images[0]
# print('After conversion --> ', image.shape)

###libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import keras
import os

In [ ]:
data_path = ('/content/drive/MyDrive/path/to/dir')
img_path= data_path
os.chdir(img_path)
print(os.path.abspath(os.getcwd()))

###libraries

In [ ]:
# import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot11.png', show_shapes=True, show_layer_names=True)

model.summary()

In [ ]:
batch_size = 16

from keras.preprocessing.image import ImageDataGenerator

# # this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False) ## cannot do flip

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(
        rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/your/dir',  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        color_mode = 'grayscale',
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/your/dir',
        target_size=(256, 256),
        color_mode = 'grayscale',
        batch_size=1,
        class_mode='binary')

# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        '/content/drive/your/dir',
        target_size=(256, 256),
        color_mode = 'grayscale',
        batch_size=1,
        class_mode=None,
        shuffle=False)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=200 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=80 // batch_size)

In [ ]:
nb_test_samples = 15
probabilities_temp = model.predict_generator(test_generator, nb_test_samples)

from sklearn.metrics import confusion_matrix
from scipy.special import expit

probabilities = expit(probabilities_temp)

y_true = np.array([0] * 13 + [1] * 2)
y_pred = probabilities

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_true, y_pred)
average_precision